In [29]:
import os
import pandas as pd

Provide information to select analysis file:

In [30]:
name_analysis = 'kmeans16'

In [34]:
cwd = os.getcwd()
directory = '..\\results\\' + name_analysis + '\\results.csv'
pth_results = os.path.join(cwd, directory)
clustering_results = pd.read_csv(pth_results)
clustering_results

,name1,name2,cluster
0,imipramine,bupropion,14
1,imipramine,hyoscyamine,7
2,imipramine,sulpiride,3
3,imipramine,clindamycin,6
4,imipramine,triamterene,13
...,...,...,...
60235,estradiol,docusate,13
60236,atropine,clarithromycin,13
60237,atropine,docusate,13
60238,clarithromycin,docusate,5


Query for a specific drug pair provide names of both drugs in cell below:

In [45]:
name1 = 'imipramine'
name2 = 'bupropion'
clustering_results[(clustering_results['name1'] == name1) & (clustering_results['name2'] == name2)]

,name1,name2,cluster
0,imipramine,bupropion,14


Provide the level of analysis you want to look at

In [48]:
analysis_level = 'soc'

In [52]:
directory = '..\\results\\' + name_analysis + '\\analysis\\' + 'scores_' + analysis_level + '_term.csv'
pth_results = os.path.join(cwd, directory)
sideeffect_results = pd.read_csv(pth_results)
sideeffect_results

,cluster,soc_term,value,tfidf_score
0,0,Blood and lymphatic system disorders,0.022472,0.093597
1,0,Cardiac disorders,0.050670,0.211038
2,0,"Congenital, familial and genetic disorders",0.001544,0.006431
3,0,Ear and labyrinth disorders,0.006159,0.025652
4,0,Endocrine disorders,0.013390,0.055769
...,...,...,...,...
411,15,"Respiratory, thoracic and mediastinal disorders",0.065343,0.271437
412,15,Skin and subcutaneous tissue disorders,0.049427,0.205322
413,15,Social circumstances,0.006538,0.027160
414,15,Surgical and medical procedures,0.010626,0.044140


Query a specific cluster

In [54]:
cluster_no = 2

In [55]:
sideeffect_results[sideeffect_results['cluster'] == cluster_no]

,cluster,soc_term,value,tfidf_score
52,2,Blood and lymphatic system disorders,0.024095,0.100481
53,2,Cardiac disorders,0.053552,0.223329
54,2,"Congenital, familial and genetic disorders",0.001835,0.007651
55,2,Ear and labyrinth disorders,0.005945,0.024791
56,2,Endocrine disorders,0.013031,0.054342
57,2,Eye disorders,0.018301,0.076321
58,2,Gastrointestinal disorders,0.076144,0.317541
59,2,General disorders and administration site cond...,0.073125,0.304951
60,2,Hepatobiliary disorders,0.016603,0.069239
61,2,Immune system disorders,0.018989,0.079190


Look at the significant rankings of each cluster at a level $\alpha$

In [56]:
analysis_level = 'soc'
alpha = '0.005'

In [58]:
directory = '..\\results\\' + name_analysis + '\\analysis\\' + 'significant_' + analysis_level + '_ranks_' + alpha + '.csv'
pth_results = os.path.join(cwd, directory)
statistical_results = pd.read_csv(pth_results)
statistical_results

,cluster,soc_term,value,tfidf_score,rank,grubbs
0,1,Vascular disorders,0.074063,0.309354,3.0,3.872983
1,10,Musculoskeletal and connective tissue disorders,0.064710,0.269116,5.0,3.063165
2,0,Musculoskeletal and connective tissue disorders,0.063239,0.263388,6.0,1.896245
3,1,Musculoskeletal and connective tissue disorders,0.059900,0.250195,7.0,0.729325


Look at the significant results summary

In [60]:
directory = '..\\results\\' + name_analysis + '\\analysis\\' + 'significant_summary.csv'
pth_results = os.path.join(cwd, directory)
summary_results = pd.read_csv(pth_results)
summary_results

,name1,name2,cluster,soc1,hlgt1,hlgt2,hlgt3,hlt1,hlt2,hlt3,pt1,pt2,pt3
0,imipramine,bupropion,14,NaN,Lower respiratory tract disorders (excl obstru...,Reproductive neoplasms male malignant and unsp...,NaN,Angioedemas,NaN,NaN,Bartter's syndrome,Congenital knee deformity,Incision site blister
1,cholecalciferol,nortriptyline,14,NaN,Lower respiratory tract disorders (excl obstru...,Reproductive neoplasms male malignant and unsp...,NaN,Angioedemas,NaN,NaN,Bartter's syndrome,Congenital knee deformity,Incision site blister
2,cholecalciferol,ceftriaxone,14,NaN,Lower respiratory tract disorders (excl obstru...,Reproductive neoplasms male malignant and unsp...,NaN,Angioedemas,NaN,NaN,Bartter's syndrome,Congenital knee deformity,Incision site blister
3,cholecalciferol,citalopram,14,NaN,Lower respiratory tract disorders (excl obstru...,Reproductive neoplasms male malignant and unsp...,NaN,Angioedemas,NaN,NaN,Bartter's syndrome,Congenital knee deformity,Incision site blister
4,cholecalciferol,perindopril,14,NaN,Lower respiratory tract disorders (excl obstru...,Reproductive neoplasms male malignant and unsp...,NaN,Angioedemas,NaN,NaN,Bartter's syndrome,Congenital knee deformity,Incision site blister
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60235,fenoterol,clarithromycin,11,NaN,NaN,NaN,NaN,Cardiac histopathology procedures,Fertility and fertilisation interventions female,NaN,Ocular pemphigoid,Sydenham's chorea,Congenital pulmonary valve disorder
60236,clonidine,digoxin,11,NaN,NaN,NaN,NaN,Cardiac histopathology procedures,Fertility and fertilisation interventions female,NaN,Ocular pemphigoid,Sydenham's chorea,Congenital pulmonary valve disorder
60237,vitamin b6,famotidine,11,NaN,NaN,NaN,NaN,Cardiac histopathology procedures,Fertility and fertilisation interventions female,NaN,Ocular pemphigoid,Sydenham's chorea,Congenital pulmonary valve disorder
60238,digoxin,famotidine,11,NaN,NaN,NaN,NaN,Cardiac histopathology procedures,Fertility and fertilisation interventions female,NaN,Ocular pemphigoid,Sydenham's chorea,Congenital pulmonary valve disorder


Query for specific drugs

In [61]:
name1 = 'imipramine'
name2 = 'bupropion'
summary_results[(summary_results['name1'] == name1) & (summary_results['name2'] == name2)]

,name1,name2,cluster,soc1,hlgt1,hlgt2,hlgt3,hlt1,hlt2,hlt3,pt1,pt2,pt3
0,imipramine,bupropion,14,NaN,Lower respiratory tract disorders (excl obstru...,Reproductive neoplasms male malignant and unsp...,NaN,Angioedemas,NaN,NaN,Bartter's syndrome,Congenital knee deformity,Incision site blister


Look at the targets for a particular level 

In [63]:
level = 'hlgt'

In [67]:
directory = '..\\results\\' + name_analysis + '\\analysis\\' + 'targets_' + level + '.csv'
pth_results = os.path.join(cwd, directory)
summary_results = pd.read_csv(pth_results, header=None)
summary_results

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0,0,0,0,0,10,10,10,10,10,...,3,3,3,3,3,6,6,6,6,6
1,DNA Polymerase,Serine Threonine Kinase,Sodium Channel,"Endopeptidase (Serine Endopeptidase, Serine Pr...",Potassium Channel,Serine Threonine Kinase,Potassium Channel,"Endopeptidase (Serine Endopeptidase, Serine Pr...",Serotonin Receptor,Sodium Channel,...,Serine Threonine Kinase,Sodium Channel,Potassium Channel,"Endopeptidase (Serine Endopeptidase, Serine Pr...",Serotonin Receptor,Serine Threonine Kinase,Potassium Channel,Sodium Channel,"Endopeptidase (Serine Endopeptidase, Serine Pr...",DNA Polymerase
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.768,1.1099999999999999,0.0,1.1280000000000001,0.377,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.913,0.903,1.424,1.43,1.361
5840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.1880000000000002,0.445,0.45,0.606,1.286,NaN,NaN,NaN,NaN,NaN
5841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.666,1.102,1.4,0.648,1.397,NaN,NaN,NaN,NaN,NaN
5842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.369,0.891,0.867,1.377,0.904
